# Punto 2 Búsqueda binaria usando índice invertido (BSII)
## Integrantes
* Juan Esteban Arboleda
* Luccas Rojas

### 1. Preprocesamiento
Lo primero que se llevara a cabo para poder hacer la busqueda binaria a travez de indice invertido es la tokenizacion de los documentos y generacion del vocabulario. Ademas es importante tener en cuenta que el vocabulario debe estar ordenado, no debe contener stop-words, debe estar stemizado y normalizado

* A continucion se cargan los documentos y los queries en una estructura de datos, se debe cambiar document_path y query_path por la ruta donde se encuentran los documentos y los queries respectivamente

In [4]:
import os
import pandas as pd

# Ruta que se debe cambiar segun la ubicacion del archivo, al descomprimir el zip no deberia afectar

documents_path = '../data/docs-raw-texts'
queries_path = '../data/queries-raw-texts'

def load_documents(folder_path):
    document = [[0,'','']]
    id = 1
    for filename in os.listdir(folder_path):
        text = pd.read_xml(os.path.join(folder_path, filename))['raw'].tolist()[1]
        filtered_text = text.replace('\n', ' ').replace('\xa0', ' ')
        row = [id, filename, filtered_text]
        document.append(row)
        id += 1
    return document

documents = load_documents(documents_path)
queries = load_documents(queries_path)

documents = pd.DataFrame(documents)
queries = pd.DataFrame(queries)

documents = documents.rename(columns = {0:'id',1:'filename',2:'text'})
queries = queries.rename(columns = {0:'id',1:'filename',2:'text'})

Primero que todo tokenizamos el texto, para esto utilizamos el word tokenize de la libreria NLTK

In [5]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

documents['tokens'] = documents['text'].apply(word_tokenize)
queries['tokens'] = queries['text'].apply(word_tokenize)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luccas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luccas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Removemos todos los signos de puntuacion, contracciones del ingles y dejamos el texto todo en minusculas (normalizar) 

In [6]:
import string

def remove_punctuation(token_list):
    return [token.lower() for token in token_list if (token not in string.punctuation and (len(token)>1 or token.isnumeric()))]

documents['tokens']=documents['tokens'].apply(lambda x: remove_punctuation(x))
queries['tokens']=queries['tokens'].apply(lambda x: remove_punctuation(x))

print(documents)

      id          filename                                               text  \
0      0                                                                        
1      1  wes2015.d001.naf  William Beaumont and the Human Digestion.  Wil...   
2      2  wes2015.d002.naf  Selma Lagerlöf and the wonderful Adventures of...   
3      3  wes2015.d003.naf  Ferdinand de Lesseps and the Suez Canal.  Ferd...   
4      4  wes2015.d004.naf  Walt Disney’s ‘Steamboat Willie’ and the Rise ...   
..   ...               ...                                                ...   
327  327  wes2015.d327.naf  James Parkinson and Parkinson’s Disease.  Wood...   
328  328  wes2015.d328.naf  Juan de la Cierva and the Autogiro.  Demonstra...   
329  329  wes2015.d329.naf  Squire Whipple – The Father of the Iron Bridge...   
330  330  wes2015.d330.naf  William Playfair and the Beginnings of Infogra...   
331  331  wes2015.d331.naf  Juan Bautista de Anza and the Route to San Fra...   

                           

Luego de tokenizar, dejar todo en minusculas, quitaremos las stop words para que reduzcan el vocabulario y no afecten el resultado final. Para esto usaremos la libreria nltk y su metodo stopwords.words('english').

In [7]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

#TODO no se si normalizar cuente como poner todo en minusculas
def remove_stop_words(token_list):
    return [token for token in token_list if token not in stop_words]

documents['tokens']=documents['tokens'].apply(lambda x: remove_stop_words(x))
queries['tokens']=queries['tokens'].apply(lambda x: remove_stop_words(x))

Luego de eliminar las stop words se hace stemming a las palabras restantes.

In [8]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
def stemming(token_list):
    return [stemmer.stem(token) for token in token_list]

documents['tokens']=documents['tokens'].apply(lambda x: stemming(x))
queries['tokens']=queries['tokens'].apply(lambda x: stemming(x))

En este punto el texto de cada documento y query esta en un formato mas facil de procesar, por lo que se procede a realizar la representacion vectorial de los documentos y queries.

## 2. Representación de los datos

A continuación se hace la implementación para transformar el anterior dataframe en una estructura de indice inertido para así poder realizar busquedas binarias 

In [9]:
inverted_index = {}

for i in range(len(documents)):
    document = documents.iloc[i]
    for token in document['tokens']:
        if token not in inverted_index:
            inverted_index[token] = {"df": 0, "postings": []}
        if i not in inverted_index[token]:
            inverted_index[token]["df"] += 1
            inverted_index[token]["postings"].append(i)
inverted_index
print(len(inverted_index))

14682


Como se pudo observar en el anterior código, se crea un diccionario que almacenara el índice invertido haciendo un recorrido por cada uno de los documentos y sus tokens. Agregando así todos los tokens del vocabulario y añadiendo a cada token el listado de documentos que contienen ese token. El vocabulario final cuenta con 14682 tokens.

## 3. Modelamiento

In [12]:
#TODO funcion para and y algoritmo de merge

def organize_tokens(tokens,inverted_index):
    organized_tokens = []
    dfs = []
    for token in tokens:
        dfs.append(inverted_index[token]['df'])     
    dfs.sort()
    for df in dfs:
        for token in tokens:
            if inverted_index[token]['df'] == df:
                organized_tokens.append(token) 
                tokens.remove(token)
    return organized_tokens   

def and_search(tokens,inverted_index):
    organized_tokens = organize_tokens(tokens,inverted_index)
    relevant_documents = []
    token_document_index = [0 for i in range(len(organized_tokens))]
    while token_document_index[0] < len(inverted_index[organized_tokens[0]]['postings']):
        document = inverted_index[organized_tokens[0]]['postings'][token_document_index[0]]
        document_is_relevant = True
        token_index = 1
        while document_is_relevant:
            document_index=0
            while token_document_index[document_index] < len(inverted_index[organized_tokens[document_index]]['postings']) and inverted_index[organized_tokens[document_index]]['postings'][token_document_index[document_index]] < document:
            if inverted_index[organized_tokens[token_index]]['postings'][array_index[token_index]] == document:
                token_index += 1
                if token_index == len(organized_tokens):
                    relevant_documents.append(document)
                    document_is_relevant = False

        token_document_index[0] += 1
        

print(and_search(queries.iloc[2]['tokens'],inverted_index))



NameError: name 'merge' is not defined